In [47]:
# Import the required libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize

In [78]:
# Import KMeans and graphic libraries
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [87]:
import folium
from geopy.geocoders import Nominatim

In [49]:
CLIENT_ID='ZOO51K0TRLO3V1BXETAEBMWHHIO2C0DXTW05UWSNGTBS5G0E'
CLIENT_SECRET='ODKWIW1VFAKDYQ1PQNNJQU1RQDCA2QMV0UNTHXJHPBCR422B'
VERSION='20181511'
LIMIT=30
RADIUS = 500

In [50]:
df_zomato =  pd.read_csv('zomato.csv',encoding='ANSI')

In [51]:
df_country_codes = pd.read_excel('Country-Code.xlsx')

In [52]:
df_country_codes.set_index('Country Code',inplace=True)

In [53]:
df_zomato.head(3)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270


In [54]:
df_zomato['Country'] = df_zomato['Country Code'].apply(lambda x : df_country_codes.loc[x,:])

In [55]:
df_zomato.head(2)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes,Country
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Yes,No,No,No,3,4.8,Dark Green,Excellent,314,Phillipines
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Yes,No,No,No,3,4.5,Dark Green,Excellent,591,Phillipines


In [56]:
df_zomato.drop(['Restaurant ID','Country Code'],inplace=True, axis=1)

In [57]:
df_zomato.head(2)

,Restaurant Name,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes,Country
0,Le Petit Souffle,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",1100,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314,Phillipines
1,Izakaya Kikufuji,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,1200,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591,Phillipines


In [58]:
cols = [df_zomato.columns[-1]] + [col for col in df_zomato if col != df_zomato.columns[-1]]

In [59]:
df_zomato = df_zomato[cols]

In [60]:
df_zomato.head(2)

,Country,Restaurant Name,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,Phillipines,Le Petit Souffle,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",1100,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,Phillipines,Izakaya Kikufuji,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,1200,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591


1. Choose a country
2. Explore the venues in and around top 5 rated restaurant

In [61]:
df_zomato.groupby('Country').count()

,Restaurant Name,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
Country,,,,,,,,,,,,,,,,,,,
Australia,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24
Brazil,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60
Canada,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
India,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652,8652
Indonesia,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
New Zealand,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40
Phillipines,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22
Qatar,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20
Singapore,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20


In [106]:
df_India = df_zomato[df_zomato['Country']=='India']

In [109]:
df_India = df_India[df_zomato['Aggregate rating']>=4.5]

c:\users\ibm_admin\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [110]:
df_India.count()

Country                 116
Restaurant Name         116
City                    116
Address                 116
Locality                116
Locality Verbose        116
Longitude               116
Latitude                116
Cuisines                116
Average Cost for two    116
Currency                116
Has Table booking       116
Has Online delivery     116
Is delivering now       116
Switch to order menu    116
Price range             116
Aggregate rating        116
Rating color            116
Rating text             116
Votes                   116
dtype: int64

In [111]:
address = 'India'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)
map_i = folium.Map(location = [latitude, longitude],zoom_start=5)
map_i

22.3511148 78.6677428


In [112]:
for lat,long,name in zip(df_India['Latitude'],df_India['Longitude'],df_India['Restaurant Name']):    
   label = '{}'.format(name)
   label = folium.Popup(label, parse_html=True)
   folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#4186FF',
        fill_opacity=0.7).add_to(map_i) 
map_i

In [115]:
df_India.groupby('City').count()

,Country,Restaurant Name,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
City,,,,,,,,,,,,,,,,,,,
Agra,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Ahmedabad,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Bangalore,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
Bhopal,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Bhubaneshwar,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
Chandigarh,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
Chennai,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
Coimbatore,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Dehradun,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [116]:
CLIENT_ID='ZOO51K0TRLO3V1BXETAEBMWHHIO2C0DXTW05UWSNGTBS5G0E'
CLIENT_SECRET='ODKWIW1VFAKDYQ1PQNNJQU1RQDCA2QMV0UNTHXJHPBCR422B'
VERSION='20181511'
LIMIT=30
RADIUS = 100

In [119]:
df_venue_all = pd.DataFrame()
venues = None
for lat,long,name  in zip(df_India['Latitude'],df_India['Latitude'],df_India['Restaurant Name']):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, RADIUS, LIMIT)
    results = requests.get(url).json()
    try:
        venues = results['response']['venues']
        df_venues = json_normalize(venues)
        #print(df_venues.shape)
        df_venues['Restaurant Name'] = name
        df_venues['Latitude'] = lat
        df_venues['Longitude'] = long
        df_venue_all = df_venue_all.append(df_venues)
    except:
        pass
    
    #print(df_venue_all.shape)

In [122]:
df_venue_all.shape

(105, 20)

In [123]:
df_venue_all.head()

,Latitude,Longitude,Restaurant Name,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,30.725704,30.725704,Barbeque Nation,"[{'id': '4f2a210c4b9023bd5841ed28', 'name': 'H...",False,525d42b011d26a607bc4ae18,NaN,EG,Kawm Ḩamādah,مصر,NaN,5068.0,"[Kom Hamada, محافظة البحيرة, مصر]","[{'label': 'display', 'lat': 30.75898933410644...",30.758989,30.761858,NaN,محافظة البحيرة,عزبة حرحش,v-1543577653
1,30.725704,30.725704,Barbeque Nation,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",False,55adee6b498ef3087182e6f2,Port Said st.,EG,Koum Hamada,مصر,NaN,4477.0,"[Port Said st., Koum Hamada, مصر]","[{'label': 'display', 'lat': 30.763039, 'lng':...",30.763039,30.708282,NaN,NaN,Traffice Station (مرور كوم حماده),v-1543577653
2,30.725704,30.725704,Barbeque Nation,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4fb05535e4b0ddb44ab812e2,Kalawat,EG,Manado,مصر,Maumbi,10067.0,"[Kalawat (Maumbi), Manado, مصر]","[{'label': 'display', 'lat': 30.76990005988812...",30.769900,30.633896,NaN,NaN,"Meeting Room, Sutan Raja Hotel",v-1543577653
3,30.725704,30.725704,Barbeque Nation,"[{'id': '5032891291d4c4b30a586d68', 'name': 'A...",False,5068cb87e4b0bb47b2e95ab7,NaN,EG,NaN,مصر,NaN,5768.0,"[Egypt, مصر]","[{'label': 'display', 'lat': 30.698676, 'lng':...",30.698676,30.674274,NaN,Egypt,Nubaria Powerstation Camp,v-1543577653
4,30.725704,30.725704,Barbeque Nation,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",False,530208fa498e9ccf0dd67de9,NaN,EG,NaN,مصر,NaN,7094.0,[مصر],"[{'label': 'display', 'lat': 30.67340677586322...",30.673407,30.683348,NaN,NaN,سجن جنوب التحرير,v-1543577653


In [124]:
def get_category(row):
    category_list =  row['categories']
    if len(category_list)==0:
        return None
    else:
        return category_list[0]['name']

In [126]:
df_venue_all['categories'] = df_venue_all.apply(get_category,axis=1)

In [130]:
df_venue_all.shape

(105, 20)

In [129]:
df_venue_all.groupby('Restaurant Name').count()

,Latitude,Longitude,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
Restaurant Name,,,,,,,,,,,,,,,,,,,
22nd Parallel,1,1,1,1,1,0,1,0,1,0,1,1,1,1,1,0,0,1,1
Aman Chicken,30,30,26,30,30,12,30,13,30,3,30,30,29,30,30,7,13,30,30
Barbeque Nation,18,18,16,18,18,4,18,6,18,1,18,18,18,18,18,0,5,18,18
Burgrill,22,22,21,22,22,4,22,6,22,1,22,22,22,22,22,2,6,22,22
La Quello - Mediterranean Kitchen,1,1,1,1,1,0,1,0,1,0,1,1,1,1,1,0,0,1,1
Mirchi And Mime,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Nawras Seafood Restaurant,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1
The Breakfast Story,1,1,1,1,1,0,1,0,1,0,1,1,1,1,1,0,0,1,1
The Great Indian Pub,30,30,28,30,30,16,30,9,30,6,30,30,30,30,30,1,12,30,30


In [131]:
url = 'https://places.factual.com/data/t/places#filters={%22$and%22:[{%22country%22:{%22$eq%22:%22US%22}},{%22category_labels%22:{%22$includes%22:%22landmarks%22}}]}&offset=000'

In [132]:
url_text = requests.get(url).text

In [133]:
soup = BeautifulSoup(url_text, 'lxml')

[<div class="min-h-full">
 <script async="async" data-cbid="b3cae2c7-6f8a-4d38-a6b9-93fbdafe3ce8" id="Cookiebot" src="https://consent.cookiebot.com/uc.js"></script>
 <header class="flex flex-wrap items-center justify-between py-2 px-4">
 <a href="https://www.factual.com/">
 <img class="w-32" src="/assets/shared/factual_logos/full_color_small-259d922390d2c005e6737b5e6a51327ce1d3b106a696603527ccb4b3800bf266.png"/>
 </a>
 <nav>
 <div class="header-item">
 <a class="header-link" href="//accounts.factual.com/login">
 Login
 </a>
 </div>
 </nav>
 </header>
 <div class="footer-push">
 <div class="container-fluid">
 <div class="row">
 <div class="col-sm-12">
 <h2>
 <span class="title">
 Global Places - 
 </span>
 <a href="/data/t/places/schema">
 Schema
 </a>
 </h2>
 </div>
 </div>
 <div class="row data-preview-interface">
 <div class="col-sm-8">
 <div class="search-and-filter">
 <div class="row">
 <form class="col-sm-8" id="search">
 <div class="input-group">
 <input id="search-query" name="q